In [15]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [16]:
# run params
SECTION = 'gan'
RUN_ID = '0001'
DATA_NAME = 'camel'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

In [17]:
total = np.load(glob('data/camel/*')[0])

# train_size = int(len(total) * 0.8)
train, test = total[:80000], total[80000:]

In [18]:
class SimpleDataset(Dataset):
    def __init__(self, npy, transform):
        self.npy = npy
        self.transform = transform

    def __len__(self):
        return len(self.npy)

    def __getitem__(self, idx):
        img = self.npy[idx]
        if self.transform:
            img = (img.astype('float32') - 127.5) / 127.5
            img = img.reshape(28, 28, 1) 
            img = self.transform(img)
            
        return img

In [19]:
NUM_CLASSES = 10
BATCH_SIZE = 64
transform = transforms.Compose([   
    transforms.ToTensor()
])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
train_dataset = SimpleDataset(train, transform)
test_dataset = SimpleDataset(test, transform)

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, BATCH_SIZE, shuffle=True)

In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 5, 2)
        self.dropout1 = nn.Dropout(0.4)

        self.conv2 = nn.Conv2d(64, 64, 5, 2)
        self.dropout2 = nn.Dropout(0.4)

        self.conv3 = nn.Conv2d(64, 128, 5, 2)
        self.dropout3 = nn.Dropout(0.4)

        self.conv4 = nn.Conv2d(128, 128, 5, 1)
        self.dropout4 = nn.Dropout(0.4)

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(2048, 1)
    def forward(self, x):
        x = F.pad(self.conv1(x), (1,1,1,1))
        x = F.relu(x)
        x = self.dropout1(x)

        x = F.pad(self.conv2(x), (1,1,1,1))
        x = F.relu(x)
        x = self.dropout2(x)

        x = F.pad(self.conv3(x), (2,2,2,2))
        x = F.relu(x)
        x = self.dropout3(x)

        x = F.pad(self.conv4(x), (1,1,1,1))
        x = F.relu(x)
        x = self.dropout4(x)
        

        x = self.flatten(x)

        x = self.linear1(x)
        return torch.sigmoid(x)

In [22]:
Discriminator()(torch.randn(1, 1, 28, 28))

tensor([[0.4976]], grad_fn=<SigmoidBackward0>)

In [23]:
z_dim = 100
class Generator(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        self.linear1 = nn.Linear(z_dim, 3136)
        self.bn1 = nn.BatchNorm1d(3136, momentum=0.9)
        self.upsample1 = nn.UpsamplingNearest2d(scale_factor=2)

        self.conv1 = nn.Conv2d(64, 128, 5, 1)
        self.bn2 = nn.BatchNorm2d(128, momentum=0.9)
        self.upsample2 = nn.UpsamplingNearest2d(scale_factor=2)

        self.conv2 = nn.Conv2d(128, 64, 5, 1)
        self.bn3 = nn.BatchNorm2d(64, momentum=0.9)

        self.convT1 = nn.ConvTranspose2d(64, 64, 5, 1, 2)
        self.bn4 = nn.BatchNorm2d(64, momentum=0.9)

        self.convT2 = nn.ConvTranspose2d(64, 1, 5, 1, 2)

        
    def forward(self, x):
        x = self.bn1(self.linear1(x))
        x = F.relu(x)
        x = x.reshape(-1, 64, 7, 7)
        x = self.upsample1(x)
        x = F.pad(self.conv1(x), (2,2,2,2))
        x = self.bn2(x)
        x = F.relu(x)

        x = self.upsample2(x)
        x = F.pad(self.conv2(x), (2,2,2,2))
        x = F.relu(self.bn3(x))

        x = self.convT1(x)
        x = self.bn4(x)
        x = self.convT2(x)

        return torch.tanh(x)

In [24]:
discriminator = Discriminator().to(device)
generator =  Generator(100).to(device)
d_optimizer = optim.RMSprop(discriminator.parameters(),lr=0.0008)
g_optimizer = optim.RMSprop(generator.parameters(), lr=0.0004)
criterion = nn.BCELoss()

In [25]:
next(iter(train_dataloader)).shape

torch.Size([64, 1, 28, 28])

In [26]:
d_losses, g_losses = [], []
for epoch in range(200):  # loop over the dataset multiple times
    g_total, d_total = 0., 0.
    for i, inputs in enumerate(train_dataloader, 0):
        
        inputs = inputs.to(device)
        real = torch.ones((BATCH_SIZE, 1)).to(device)
        fake = torch.zeros((BATCH_SIZE, 1)).to(device)
        generator.eval()
        generated_images = generator(torch.randn((BATCH_SIZE, z_dim)).to(device))

        d_loss = criterion(discriminator(generated_images), fake)
        d_loss += criterion(discriminator(inputs), real)

        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        generator.train()
        discriminator.eval()
        real = torch.ones((BATCH_SIZE, 1)).to(device)
        generated_images = generator(torch.randn((BATCH_SIZE, z_dim)).to(device))

        g_loss = criterion(discriminator(generated_images), real)
        
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        d_total += d_loss.item()
        g_total += g_loss.item()
    
    d_losses.append(d_total/len(train_dataloader))
    g_losses.append(g_total/len(train_dataloader))

    print(f'[{epoch + 1}] d_loss: {d_total / len(train_dataloader):.3f} g_loss: {g_total / len(train_dataloader):.3f}')
    torch.save(discriminator, RUN_FOLDER + "/weights/discriminator.pt")
    torch.save(generator, RUN_FOLDER + "/weights/generator.pt")


print('Finished Training')

In [ ]:
fig = plt.figure()
plt.plot([x for x in d_losses], color='black', linewidth=0.25)

plt.plot([x for x in g_losses], color='orange', linewidth=0.25)

plt.show()


NameError: name 'd_losses' is not defined

<Figure size 432x288 with 0 Axes>